# Mercado imobiliário de Fortaleza (2020)
### Apartamentos à venda — análise exploratória, mapa e modelo preditivo
---


## Introdução
O ano de 2020 começou sob forte incerteza com a pandemia de COVID-19, afetando renda e confiança. Ainda assim, juros em patamares historicamente baixos mantiveram o mercado imobiliário ativo. Neste estudo, analisamos anúncios de apartamentos à venda em Fortaleza (junho/2020) para obter uma visão clara do mercado e responder perguntas práticas.


## Objetivos
- **Quantificar** a oferta de apartamentos por bairro.
- **Identificar** bairros mais valorizados (preço/m²).
- **Entender** fatores que influenciam preço (m², quartos, banheiros, vagas).
- **Construir** um modelo simples para estimar preço a partir dos atributos.


## Dados e metodologia
- **Fonte**: Web scraping de portais de imóveis (arquivo `datasets/imoveis_junho.csv`).
- **Geografia**: GeoJSON oficial de bairros de Fortaleza.
- **Etapas**: carregamento, limpeza, EDA, mapa coroplético e modelagem (Random Forest).


In [1]:
# Imports e configuração
import warnings
warnings.filterwarnings('ignore')

import re
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

%matplotlib inline
pd.set_option('display.max_colwidth', 200)


In [2]:
# Carregamento
DATA_PATH = 'datasets/imoveis_junho.csv'
GEOJSON_PATH = 'datasets/FortalezaBairros.geojson'

df = pd.read_csv(DATA_PATH)
df.shape


(3120, 7)

## Limpeza dos dados
Passos aplicados:
- Remoção de registros sem preço ou com valores *sob consulta*.
- Exclusão de linhas com intervalos (ex.: `48 - 68`) para garantir consistência numérica.
- Conversão de `price`, `area_m2`, `bedrooms`, `bathrooms`, `vagas` para numéricos.
- Remoção de valores ausentes e duplicados.
- Corte de valores irreais (quartos/banheiros/vagas > 7).
- Criação de `price_per_m2`.


In [3]:
# Remover preços 'Sob consulta' e linhas com intervalos em colunas-chave
df = df[df['price'].astype(str).str.lower().str.contains('sob consulta') == False]

def has_range(x):
    s = str(x)
    return '-' in s

for col in ['area_m2','bedrooms','bathrooms','vagas']:
    df = df[~df[col].apply(has_range)]

# Coerção para numérico
num_cols = ['price','area_m2','bedrooms','bathrooms','vagas']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Remover ausentes e duplicados
df = df.dropna(subset=num_cols + ['endereco']).drop_duplicates().copy()

# Cortes conservadores para outliers estruturais
df = df[(df['bedrooms'] <= 7) & (df['bathrooms'] <= 7) & (df['vagas'] <= 7)]

# Preço por m2
df['price_per_m2'] = (df['price'] / df['area_m2']).round(2)
df.shape


(2759, 8)

### Extração e padronização de bairros
- Extraímos `Bairro` a partir da coluna `endereco`.
- Padronizamos para maiúsculas, sem acento e alinhamos nomenclaturas ao GeoJSON oficial.


In [4]:
street_tokens = {'RUA','AVENIDA','TRAVESSA','ALAMEDA','AV','AL'}

def extract_bairro(endereco: str) -> str:
    parts = [p.strip() for p in str(endereco).split(',')]
    if len(parts) >= 2:
        first_token = parts[0].split()[0].upper() if parts[0] else ''
        return parts[1] if first_token in street_tokens else parts[0]
    return parts[0] if parts else np.nan

df['Bairro'] = df['endereco'].apply(extract_bairro).astype(str)
df['Bairro'] = df['Bairro'].apply(lambda x: unidecode(x).upper().strip())

# Ajustes de nomenclatura para casar com o GeoJSON
name_fix = {
    'PREFEITO JOSE WALTER': 'PREFEITO JOSE VALTER',
    'MANOEL DIAS BRANCO': 'MANUEL DIAS BRANCO',
    'ENGENHEIRO LUCIANO CAVALCANTE': 'ENG LUCIANO CAVALCANTE',
    'SAPIRANGA': 'SAPIRANGA COITE',
    'GUARARAPES': 'PATRIOLINO RIBEIRO'
}
df['Bairro'] = df['Bairro'].replace(name_fix)

# Remover registros genéricos
df = df[df['Bairro'] != 'FORTALEZA'].copy()
df.shape


(2659, 9)

## EDA — visão geral
- Bairros com pelo menos 7 anúncios.
- Distribuição de preços e boxplot por bairro (com corte em R$ 1,1 mi para reduzir *outliers* extremos).
- Correlações entre variáveis numéricas.


In [5]:
# Bairros com frequência mínima
min_ads = 7
b_counts = df['Bairro'].value_counts()
b_sel = b_counts[b_counts >= min_ads].index
df_sel = df[df['Bairro'].isin(b_sel)].copy()

# Corte de preço para reduzir outliers extremos
df_sel = df_sel[df_sel['price'] < 1_100_000].copy()

# Gráficos (renderização interativa no notebook)
fig_counts = px.bar(b_counts.loc[b_sel].sort_values(ascending=False),
                     title='Quantidade de apartamentos por bairro (>=7 anúncios)',
                     labels={'value':'Quantidade','index':'Bairro'})

fig_box = px.box(df_sel, x='Bairro', y='price',
                 title='Distribuição de preços por bairro',
                 category_orders={'Bairro': list(b_sel)})
fig_box.update_layout(xaxis_tickangle=45, height=600)

fig_hist_price = px.histogram(df_sel, x='price', nbins=20, title='Distribuição de preços')

fig_counts.show(); fig_box.show(); fig_hist_price.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Correlação
num_df = df_sel[['price','area_m2','bedrooms','bathrooms','vagas','price_per_m2']].copy()
corr = num_df.corr()
plt.figure(figsize=(12,5))
sns.heatmap(corr, cmap='coolwarm_r', linewidths=0.5, annot=True)
plt.title('Correlação entre variáveis numéricas')
plt.show()


## Bairros mais valorizados (preço/m²)
Calculamos médias por bairro e ranqueamos por `price_per_m2`. Também exibimos a quantidade de anúncios que compõem cada média.


In [ ]:
bairros_stats = (df_sel.groupby('Bairro', as_index=False)
                 .agg(price=('price','mean'),
                      area_m2=('area_m2','mean'),
                      bedrooms=('bedrooms','mean'),
                      bathrooms=('bathrooms','mean'),
                      vagas=('vagas','mean'),
                      price_per_m2=('price_per_m2','mean')))
bairros_stats['Number Ads'] = bairros_stats['Bairro'].map(b_counts)
bairros_stats = bairros_stats.round(2).sort_values('price_per_m2', ascending=False)

fig_bar_m2 = px.bar(bairros_stats, x='Bairro', y='price_per_m2', color='price_per_m2',
                    color_continuous_scale='Portland',
                    title='Média de preço por m² por bairro',
                    hover_data=['Number Ads'])
fig_bar_m2.update_layout(xaxis_tickangle=45, height=600)
fig_bar_m2.show()


## Mapa — coroplético por preço/m²
Integramos as médias por bairro ao GeoJSON oficial para mapear o preço/m². A legenda indica faixas de valor; o *tooltip* mostra bairro, preço/m² e número de anúncios.


In [ ]:
# GeoJSON
geo = json.load(open(GEOJSON_PATH))
gdf = gpd.GeoDataFrame.from_features(geo, crs='epsg:4326')
gdf = gdf.rename(columns={'NOME':'Bairro'})[['Bairro','geometry']]
gdf['Bairro'] = gdf['Bairro'].astype(str)

# Merge
merged = gdf.merge(bairros_stats[['Bairro','price_per_m2','Number Ads']], on='Bairro', how='left')

# Mapa Folium
center = [-3.7658, -38.5078]
m = folium.Map(location=center, tiles='cartodbpositron', zoom_start=12)

ch = folium.Choropleth(
    geo_data=merged,
    data=merged,
    columns=['Bairro','price_per_m2'],
    key_on='feature.properties.Bairro',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.4,
    nan_fill_color='lightgray',
    legend_name='Preço por m² (R$)'
).add_to(m)

ch.geojson.add_child(folium.features.GeoJsonTooltip(
    fields=['Bairro','price_per_m2','Number Ads'],
    aliases=['Bairro:','Preço/m²:','Anúncios:'],
    localize=True
))
m


## Modelo preditivo — Random Forest
Modelo simples para estimar preço a partir de m², quartos, banheiros, vagas e bairro (one-hot). Reportamos R² e MAE, além das importâncias das variáveis.


In [ ]:
# One-hot de bairro
df_model = df_sel.drop(columns=['price_per_m2']).copy()
df_model = df_model.join(pd.get_dummies(df_model['Bairro'], prefix='', prefix_sep=''))
X = df_model.drop(columns=['price','Bairro'])
y = df_model['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print({'r2': round(r2,4), 'mae': round(mae,2)})

# Importância de variáveis (top 20)
imp = (pd.Series(rf.feature_importances_, index=X.columns)
       .sort_values(ascending=False).head(20))
fig_imp = px.bar(imp[::-1], orientation='h', title='Importância das variáveis (top 20)')
fig_imp.show()


## Conclusões
- **Oferta** concentrada em bairros com infraestrutura consolidada.
- **Preço/m²** mais alto em regiões de orla e áreas premium (ex.: Mucuripe, Praia de Iracema, Eng. Luciano Cavalcante).
- **Determinantes**: área (m²) aparece como a variável mais relevante, seguida de localização (bairro) e quantidade de banheiros/vagas.
- **Modelo**: Random Forest apresenta desempenho razoável para estimativas iniciais (acompanhe R²/MAE reportados).

### Próximos passos
- Enriquecer dados (andar, idade do prédio, condomínio, vagas cobertas, lazer).
- Tratar ranges capturando valores mínimos/médios, em vez de filtrar.
- Avaliar modelos lineares regularizados e gradiente boosting.
- Publicar painel interativo (Dash/Streamlit) com filtros por bairro.
